# Trying Bioruby
test the following:

In [1]:
require 'bio'

true

In [2]:
seq = Bio::Sequence::NA.new("atgcatgcaaaa")

"atgcatgcaaaa"

In [3]:
seq.complement

"ttttgcatgcat"

# Working with nucleic / amino acid sequences (Bio::Sequence class)
The Bio::Sequence class allows the usual sequence transformations and translations. In the example below the DNA sequence "atgcatgcaaaa" is converted into the complemental strand and spliced into a subsequence; next, the nucleic acid composition is calculated and the sequence is translated into the amino acid sequence, the molecular weight calculated, and so on. When translating into amino acid sequences, the frame can be specified and optionally the codon table selected (as defined in codontable.rb).

In [4]:
seq = Bio::Sequence::NA.new("atgcatgcaaaa")

"atgcatgcaaaa"

In [5]:
# complemental sequence (Bio::Sequence::NA object)
seq.complement

"ttttgcatgcat"

In [6]:
seq.subseq(3,8) # gets subsequence of positions 3 to 8 (starting from 1)

"gcatgc"

In [7]:
seq.gc_percent

33

In [8]:
seq.composition

{"a"=>6, "t"=>2, "g"=>2, "c"=>2}

In [9]:
seq.translate

"MHAK"

In [10]:
seq.translate(2)        # translate from frame 2

"CMQ"

In [11]:
seq.translate(1,11)     # codon table 11

"MHAK"

In [12]:
seq.translate.codes

["Met", "His", "Ala", "Lys"]

In [13]:
seq.translate.names

["methionine", "histidine", "alanine", "lysine"]

In [14]:
seq.translate.composition

{"M"=>1, "H"=>1, "A"=>1, "K"=>1}

In [15]:
seq.translate.molecular_weight

485.6050000000001

In [16]:
seq.complement.translate

"FCMH"

get a random sequence with the same NA count:

In [17]:
counts = {'a'=>seq.count('a'),'c'=>seq.count('c'),'g'=>seq.count('g'),'t'=>seq.count('t')}

{"a"=>6, "c"=>2, "g"=>2, "t"=>2}

In [18]:
randomseq = Bio::Sequence::NA.randomize(counts)

"ctaataagaagc"

Nucleic acid sequence are members of the Bio::Sequence::NA class, and amino acid sequence are members of the Bio::Sequence::AA class. Shared methods are in the parent Bio::Sequence class.

As Bio::Sequence inherits Ruby's String class, you can use String class methods. For example, to get a subsequence, you can not only use subseq(from, to) but also String#[].

Please take note that the Ruby's string's are base 0 - i.e. the first letter has index 0, for example:

In [21]:
s = 'abc'

"abc"

In [22]:
s[0].chr

"a"

In [23]:
s[0..1]

"ab"

So when using String methods, you should subtract 1 from positions conventionally used in biology. (subseq method will throw an exception if you specify positions smaller than or equal to 0 for either one of the "from" or "to".)

The window_search(window_size, step_size) method shows a typical Ruby way of writing concise and clear code using 'closures'. Each sliding window creates a subsequence which is supplied to the enclosed block through a variable named +s+.

- Show average percentage of GC content for 20 bases (stepping the default one base at a time):

In [24]:
seq = Bio::Sequence::NA.new("atgcatgcaattaagctaatcccaattagatcatcccgatcatcaaaaaaaaaa")

"atgcatgcaattaagctaatcccaattagatcatcccgatcatcaaaaaaaaaa"

In [26]:
a=[]; seq.window_search(20) { |s| a.push s.gc_percent }
a

[30, 35, 40, 40, 35, 35, 35, 30, 25, 30, 30, 30, 35, 35, 35, 35, 35, 40, 45, 45, 45, 45, 40, 35, 40, 40, 40, 40, 40, 35, 35, 35, 30, 30, 30]

Since the class of each subsequence is the same as original sequence (Bio::Sequence::NA or Bio::Sequence::AA or Bio::Sequence), you can use all methods on the subsequence. For example,

- Shows translation results for 15 bases shifting a codon at a time


In [27]:
a = []
seq.window_search(15, 3) { | s | a.push s.translate }
a

["MHAIK", "HAIKL", "AIKLI", "IKLIP", "KLIPI", "LIPIR", "IPIRS", "PIRSS", "IRSSR", "RSSRS", "SSRSS", "SRSSK", "RSSKK", "SSKKK"]

Finally, the window_search method returns the last leftover subsequence. This allows for example

- Divide a genome sequence into sections of 10000bp and output FASTA formatted sequences (line width 60 chars). The 1000bp at the start and end of each subsequence overlapped. At the 3' end of the sequence the leftover is also added:

In [28]:
i = 1
textwidth=60
remainder = seq.window_search(10000, 9000) do |s|
  puts s.to_fasta("segment #{i}", textwidth)
  i += 1
end
if remainder
  puts remainder.to_fasta("segment #{i}", textwidth) 
end

If you don't want the overlapping window, set window size and stepping size to equal values.

Other examples

- Count the codon usage

In [29]:
codon_usage = Hash.new(0)
seq.window_search(3, 3) { |s| codon_usage[s] += 1 }
codon_usage

{"atg"=>1, "cat"=>1, "gca"=>1, "att"=>2, "aag"=>1, "cta"=>1, "atc"=>1, "cca"=>1, "aga"=>1, "tca"=>3, "tcc"=>1, "cga"=>1, "aaa"=>3}

- Calculate molecular weight for each 10-aa peptide (or 10-nt nucleic acid)

In [30]:
a = []
seq.window_search(10, 10) { |s| a.push s.molecular_weight }
a

[3096.2061999999996, 3086.1961999999994, 3056.1762, 3023.1261999999997, 3073.2262]